<a href="https://colab.research.google.com/github/wsparrow/FinanceDataAnalysis-FC/blob/master/2_Project_team2_FC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dart 공시발표가 주가에 미치는 영향(실적발표 중심)


In [0]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize

auth = 'xxx' # 인증키

##1. 기업개황(예: kakao[035720])
- 기업개황 전체

In [31]:
url_tmpl = 'http://dart.fss.or.kr/api/company.json?auth={auth}&crp_cd={code}'
url = url_tmpl.format(auth=auth, code='035720')
r = requests.get(url)

jo = json.loads(r.text)
jo

{'acc_mt': '12',
 'adr': '제주특별자치도 제주시 첨단로 242',
 'bsn_no': '1208147521',
 'ceo_nm': '여민수, 조수용',
 'crp_cls': 'Y',
 'crp_nm': '(주)카카오',
 'crp_nm_e': 'Kakao Corp.',
 'crp_nm_i': '카카오',
 'crp_no': '1101111129497',
 'err_code': '000',
 'err_msg': '정상',
 'est_dt': '19950216',
 'fax_no': '02-6003-5401',
 'hm_url': 'www.kakaocorp.com',
 'ind_cd': '63120',
 'ir_url': '',
 'phn_no': '02-6718-1082',
 'stock_cd': '035720'}

- 사업자등록번호?

In [32]:
jo['bsn_no'] # bsn_no = 사업자등록번호

'1208147521'

## 2. 정보검색
- kakao의 전체 사업+반기+분기보고서 100건

In [33]:
url_tmpl = 'http://dart.fss.or.kr/api/search.json?auth={auth}&crp_cd={code}&start_dt=19990101&bsn_tp=A001&bsn_tp=A002&bsn_tp=A003&page_set=100'
url = url_tmpl.format(auth=auth, code='035720')
r = requests.get(url)
jo = json.loads(r.text)
result = json_normalize(jo, 'list')
print('총', len(result), '건')
result.head()


총 86 건


,crp_cd,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rmk,rpt_nm
0,035720,Y,카카오,카카오,20190515,20190515002549,,분기보고서 (2019.03)
1,035720,Y,카카오,카카오,20190401,20190401005013,연,사업보고서 (2018.12)
2,035720,Y,카카오,카카오,20181204,20181204000182,,[기재정정]분기보고서 (2018.09)
3,035720,Y,카카오,카카오,20181204,20181204000180,,[기재정정]반기보고서 (2018.06)
4,035720,Y,카카오,카카오,20181114,20181114002663,정,분기보고서 (2018.09)


- 보고서  url

In [34]:
df = result[:10] # 처음 10개만

for ix, row in df.iterrows():
    url_tmpl = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo={}"
    url = url_tmpl.format(row['rcp_no'])
    print (ix, url)

0 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20190515002549
1 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20190401005013
2 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20181204000182
3 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20181204000180
4 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20181114002663
5 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180814002960
6 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180515002511
7 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180330002462
8 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20171114002088
9 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170814002090


## 3. 데이터 추출

In [0]:
import re
import urllib
from bs4 import BeautifulSoup
from datetime import datetime

def sub_report_urls(rcp_no):
  '''
  접수번호(rcp_no)에 해당하는 모든 하위 보고서 URL을 추출하여 리스트로 반환
  '''
  doc_urls = []
  url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=%s" % (rcp_no)
  r = requests.get(url)

  multi_page_re = "{viewDoc\('(\d+)', '(\d+)', '(\d+)', '(\d+)', '(\d+)', '(\S+)'\)\;"
  single_page_re = "\t\tviewDoc\('(\d+)', '(\d+)', '(\d+)', '(\d+)', '(\d+)', '(\S+)'\)\;"
  
  matches = re.findall(multi_page_re, r.text)
  if len(matches) == 0: 
    matches = re.findall(single_page_re, r.text)

  doc_url_tmpl = "http://dart.fss.or.kr/report/viewer.do?rcpNo=%s&dcmNo=%s&eleId=%s&offset=%s&length=%s&dtd=%s" 
  for m in matches:
    url = doc_url_tmpl % m
    doc_urls.append(url)
  return doc_urls


def sub_report_titles(rcp_no):
    '''
    접수번호(rcp_no)에 해당하는 모든 하위 보고서의 제목 리스트 반환
    '''
    url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=%s" % (rcp_no)
    r = requests.get(url)
    matches = re.findall('text: \"(.*)\",', r.text)
    
    doc_titles = matches[1:] # '전체' 제외
    if len(doc_titles) == 0: # 1페이지 경우 (본문의 첫 라인)
      title = BeautifulSoup(r.text).title.text.strip()
      doc_titles = [title]
   
    return doc_titles

In [38]:
doc_urls = sub_report_urls('20190515002549')

print('하위보고서 갯수:', len(doc_urls))

for url in doc_urls:
    print (url)

하위보고서 갯수: 33
http://dart.fss.or.kr/report/viewer.do?rcpNo=20190515002549&dcmNo=6741214&eleId=1&offset=658&length=3918&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20190515002549&dcmNo=6741214&eleId=2&offset=4656&length=410&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20190515002549&dcmNo=6741214&eleId=3&offset=5070&length=223612&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20190515002549&dcmNo=6741214&eleId=4&offset=5160&length=106687&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20190515002549&dcmNo=6741214&eleId=5&offset=111851&length=44277&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20190515002549&dcmNo=6741214&eleId=6&offset=156132&length=36620&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20190515002549&dcmNo=6741214&eleId=7&offset=192756&length=21687&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20190515002549&dcmNo=6741214&eleId=8&offset=214447&length=5880&dtd=dart3.xsd
http://dart.fs